In [86]:
import pandas as pd
import ast
import numpy as np
import re 
import sys
import os
import pathlib
import glob
import cv2 as cv
import importlib
from scipy.spatial.transform import Rotation as R

sys.path.append("../../")
from data_processor import readCSVWStrArray
from data_processor import IntrinsicCalib
from data_processor import timeStampAligner
from data_processor import videoToPng

In [50]:
importlib.reload(timeStampAligner)

<module 'data_processor.timeStampAligner' from '/home/fj/Projects/ARPA-H/Scripts/20241126_hand_eye_pivot_calibrations/../../data_processor/timeStampAligner.py'>

In [51]:
## Clean/Align the tracking data
basePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations")
timeStampAligner.processDirectory(basePath)

Found 4 NDI files and 1 video timestamp files.
Processing NDI file: /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/ndi_hand_eye_test_1.csv
Processing video timestamp file: /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/hand_eye_test_2.csv
(9381, 1, 4, 4)
Aligned data saved to /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/ndi_hand_eye_test_1_aligned.csv
Processing NDI file: /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/ndi_hand_eye_test_1_COPY.csv
Processing video timestamp file: /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/hand_eye_test_2.csv
(9381, 1, 4, 4)
Aligned data saved to /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/ndi_hand_eye_test_1_COPY_aligned.csv
Processing NDI file: /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/ndi_hand_eye_test_2.csv
Processing video timestamp file: /media/fj/Data/Pro

In [57]:
videoBasePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations")
videoPath = videoBasePath / "hand_eye_test_2.avi"
imageBasePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations//Recordings/run_2/cam0")
imagePath = imageBasePath / "data"
timeStampsPath = videoBasePath / "hand_eye_test_2.csv"
timeStamps = np.genfromtxt(timeStampsPath, delimiter=",", dtype=np.double)[:, 1]

ndiDatapath = videoBasePath / "ndi_hand_eye_test_2_aligned.csv"



In [25]:
# Define the dimensions of the checkerboard (number of inner corners per row and column)
CHECKERBOARD_SHAPE = (10, 7)  # Adjust according to your checkerboard dimensions
CHECKERBOARD_GRID_SIZE = 5 # mm

In [30]:
videoToPng.saveFramesAsPng(videoPath, imagePath, timeStamps=timeStamps) # newSize=(640, 360), 


10868, 1732652848813

In [ ]:
# Camera Calibration
images = glob.glob(str(imagePath / "*.png"))
print(f"# of Images: {len(images)}")
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
ret, camera_matrix, dist_coeffs, rvecs, tvecs, fileNames = IntrinsicCalib.detectCheckerboardAndIntrinsicCalib(images, CHECKERBOARD_SHAPE=CHECKERBOARD_SHAPE, CHECKERBOARD_GRID_SIZE=20, savePath=imageBasePath / "intrinsics", visualize=True, imageRange=[2500, len(images)-1000, 20], returnUsedFileNames=True)

# of Images: 10869
9880/10869
Error:
 0.9313842907736611
Camera Matrix:
 [[522.85179215   0.         680.9055524 ]
 [  0.         524.49884771 361.78848927]
 [  0.           0.           1.        ]]

Distortion Coefficients:
 [[-0.38779207  0.20664283  0.00156769  0.00102807 -0.08947456]]


In [98]:
usedTimeStamps = [int(f.split("/")[-1][:-4]) for f in fileNames]
R_target2cam = rvecs
t_target2cam = tvecs

ndiData = np.genfromtxt(ndiDatapath, delimiter=",") # [Translation, Quat]
ndiDataTimeStamps = ndiData[:, 0].astype(int)

usedNdiData = []
for usedTimeStamp in usedTimeStamps:
    idx = np.where(ndiDataTimeStamps == usedTimeStamp)[0][0]
    usedNdiData.append(ndiData[idx, :])

usedNdiData = np.array(usedNdiData)
t_gripper2base = [usedNdiData[i, 1:4] for i in range(usedNdiData.shape[0])]
R_gripper2base = [R.from_quat(usedNdiData[i, 4:]).as_matrix() for i in range(usedNdiData.shape[0])]

In [113]:
handEyeR, handEyet = cv.calibrateHandEye(R_gripper2base, t_gripper2base, R_target2cam, t_target2cam)

#### Hand-Eye Calibration Result
Using Method from [Tsai](https://ieeexplore.ieee.org/document/34770)
Other OpenCV implemented methods are listed [here](https://docs.opencv.org/4.5.4/d9/d0c/group__calib3d.html#gad10a5ef12ee3499a0774c7904a801b99).

Rotation and Translation
```
(array([[-0.05006572, -0.46495374,  0.88391823],
        [ 0.01838049, -0.88530718, -0.46464326],
        [ 0.99857678, -0.00701585,  0.05286962]]),
 array([[ 83.64166892],
        [ 77.06542646],
        [-21.23159487]]))
```





In [115]:
handEyeT = np.concatenate((handEyeR, handEyet), axis=-1)
print(handEyeT)
handEyeT = np.concatenate((handEyeT, np.array([[0, 0, 0, 1]])), axis=0)

[[-5.00657160e-02 -4.64953741e-01  8.83918233e-01  8.36416689e+01]
 [ 1.83804919e-02 -8.85307176e-01 -4.64643262e-01  7.70654265e+01]
 [ 9.98576778e-01 -7.01584563e-03  5.28696162e-02 -2.12315949e+01]]


In [117]:
np.save("./handEyeOutput.npy", handEyeT)